# Meal Allowance Optimization: Price, Inflation, Prediction

## **Introduction**

In any organization, employee satisfaction is directly tied to their well-being, which encompasses physical, emotional, and financial health. As part of employee welfare, providing adequate meal allowances is one of the primary considerations. The Philippine workplace context, particularly in private sector companies such as MotorPH, often faces challenges in ensuring that employees receive sufficient allowances that reflect their needs while being sustainable for the company.

This study explores the feasibility of implementing a **meal allowance system** at MotorPH, aiming to balance **employee well-being**, **inflation adjustments**, and the **company’s budgetary constraints**. We propose a **Meal Price Model** to standardize meal allowance, leveraging historical price data and employee salary information. We also consider adjusting for the **Cost of Living** (COLA) and **inflation** to ensure that the meal allowance remains relevant over time.

## **Objectives**

The primary objectives of this project are:
1. **To develop a Meal Price Model** based on common meal components and the costs of essential commodities (e.g., proteins, carbohydrates, vegetables, fruits).
2. **To estimate the range of appropriate meal allowances** for MotorPH employees based on employee salaries, adjusting for factors such as inflation and the cost of living.
3. **To propose three meal allowance options** that balance employee needs with the company's budget. The options will take into account employee well-being, inflation factors, and fair compensation based on salary bands.

In [7]:
# Load Libraries
library(ggplot2)
library(dplyr)
library(lubridate) 
library(RColorBrewer)
library(viridis)
library(tidyr)  

In [8]:
options(repr.plot.width = 14, repr.plot.height = 11)
# Load CSV
food_prices <- read.csv("food_prices_ph_cleaned.csv", stringsAsFactors = TRUE)
employees <- read.csv("employees_details_cleaned.csv", stringsAsFactors = TRUE)

# Convert Date to date format
food_prices$Date <- as.Date(food_prices$Date)

# Ensure that numeric columns are properly formatted
food_prices$Price <- as.numeric(food_prices$Price)

# Extract year
food_prices$Year <- year(food_prices$Date)


# Filter for dates from 2019 onwards and exclude Unit entries
food_prices <- food_prices %>%
  filter(year(Date) >= 2019, Unit != "Unit")


# Exclude Unit entries
food_prices <- food_prices %>%
  filter(Unit != "Unit")


### Option 1: Meal Price Model Development